In [ ]:
import numpy as np
import brainlit
import scipy
from brainlit.utils import swc
from cloudvolume.exceptions import SkeletonDecodeError
from brainlit.algorithms.trace_analysis.fit_spline import GeometricGraph
from brainlit.algorithms.trace_analysis.spline_fxns import curvature, torsion
import os
from pathlib import Path
import pandas as pd
from networkx.readwrite import json_graph
import json
import matplotlib.pyplot as plt
import seaborn as sns

## Define brain, find trace data folder

In [ ]:
#specify brain1 or brain2 below
brain = "brain2"

root_dir = Path(os.path.abspath('')).parents[1]
experiment_dir = os.path.join(root_dir, "axon_geometry")
data_dir = os.path.join(experiment_dir, "data", brain)
segments_swc_dir = os.path.join(data_dir, "segments_swc")
trace_data_dir = os.path.join(data_dir, "trace_data")
trace_data_dir = os.path.join(trace_data_dir, "14")
print(f"Directory where swcs reside: {segments_swc_dir}")

## Read trace data

In [ ]:
max_id = 300
seg_lengths = []
mean_torsions = []
mean_curvatures = []
d_from_root = []
torsions = []
curvatures = []

def classify_height(row):
    height = row["height"]
    if height <= 2:
        return height
    else:
        return 3

df_path = os.path.join(trace_data_dir, "df.csv")
if os.path.exists(df_path):
    df = pd.read_csv(df_path)
else:
    df = pd.DataFrame(columns=["seg_id", "height", "log_segment_length", "measure", "log_value"])
    for i in np.arange(0, max_id):
        i = int(i)
        trace_data_path = os.path.join(trace_data_dir, "{}.npy".format(i))
        if os.path.exists(trace_data_path) is True:
            trace_data = np.load(trace_data_path, allow_pickle=True)
            print("Loaded segment {}".format(i))

            for node in trace_data:
                # print(node)
                seg_length = node["seg_length"]
                height = node["height"]
                mean_curvature = node["mean_curvature"]
                mean_torsion = node["mean_torsion"]
                log_seg_length = np.log10(seg_length)
                if mean_curvature > 1e-16:
                    log_mean_curvature = np.log10(mean_curvature)
                    df = df.append({"seg_id": i, "height": height, "log_seg_length": log_seg_length, "measure": "curvature", "log_value": log_mean_curvature}, ignore_index=True)
                if mean_torsion > 1e-16:
                    log_mean_torsion = np.log10(mean_torsion)
                    df = df.append({"seg_id": i, "height": height, "log_seg_length": log_seg_length, "measure": "torsion", "log_value": log_mean_torsion}, ignore_index=True)
    df.to_csv(df_path)
df["height_class"] = df.apply(classify_height, axis=1)

# Density of `log` segment lengths as a function of spline height

In [ ]:
sns.set_theme()
sns.set_context("paper")

g = sns.FacetGrid(df, col="measure", hue="height", height=4)
g.map(sns.kdeplot, "log_seg_length", bw_adjust=1.8)
g.set_axis_labels(r"$\log$ segment length", "density")
g.add_legend(title="Spline height")
plt.subplots_adjust(top=0.9)
g.fig.suptitle("Brain 1" if brain=="brain1" else "Brain 2")
g.savefig(os.path.join(experiment_dir, "figures",  f"{brain}_seg_length_height_density.eps"))
g.savefig(os.path.join(experiment_dir, "figures", f"{brain}_seg_length_height_density.jpg"))

# Density of `log` segment lengths as a function of spline height class

In [ ]:
sns.set_theme()
sns.set_context("paper")

g = sns.FacetGrid(df, col="measure", hue="height_class", height=4)
g.map(sns.kdeplot, "log_seg_length", bw_adjust=1.8)
g.set_axis_labels(r"$\log$ segment length", "density")
g.add_legend(title="Spline height class")
plt.subplots_adjust(top=0.9)
g.fig.suptitle("Brain 1" if brain=="brain1" else "Brain 2")
g.savefig(os.path.join(experiment_dir, "figures",  f"{brain}_seg_length_height_class_density.eps"))
g.savefig(os.path.join(experiment_dir, "figures", f"{brain}_seg_length_height_class_density.jpg"))

# Density of `log` curvature/torsion as a function of spline height class

In [ ]:
g = sns.FacetGrid(df, col="measure", hue="height_class", height=4)
g.map(sns.kdeplot, "log_value")
g.set_axis_labels(r"$\log$ value", "density")
g.add_legend(title="Spline height class")
g.set(xlim=[-3, 0])
plt.subplots_adjust(top=0.9)
g.fig.suptitle("Brain 1" if brain=="brain1" else "Brain 2")
g.savefig(os.path.join(experiment_dir, "figures",  f"{brain}_mean_value_height_class_density.eps"))
g.savefig(os.path.join(experiment_dir, "figures", f"{brain}_mean_value_height_class_density.jpg"))

# Scatter plots of `log` curvature/torsion as a function of spline height class

In [ ]:
g = sns.FacetGrid(df, row="measure", col="height_class", hue="height_class", height=4)
g.map(sns.scatterplot, "log_seg_length", "log_value", marker='.', s=50)
g.set_axis_labels(r"$\log$ segment length", r"$\log$ mean value")
g.set(ylim=[-4, 1])
plt.subplots_adjust(top=0.9)
g.fig.suptitle("Brain 1" if brain=="brain1" else "Brain 2")
g.savefig(os.path.join(experiment_dir, "figures",  f"{brain}_mean_value_height_class_scatter.eps"))
g.savefig(os.path.join(experiment_dir, "figures", f"{brain}_mean_value_height_class_scatter.jpg"))

# Linear regression of `log` curvature/torsion as a function of spline height class

In [ ]:
g = sns.FacetGrid(df, row="measure", col="height_class", hue="height_class", height=4)
g.map(sns.regplot, "log_seg_length", "log_value", marker='.', scatter_kws={'s':1})
g.set_axis_labels(r"$\log$ segment length", r"$\log$ mean value")
g.set(ylim=[-4, 1])
plt.subplots_adjust(top=0.9)
g.fig.suptitle("Brain 1" if brain=="brain1" else "Brain 2")
g.savefig(os.path.join(experiment_dir, "figures",  f"{brain}_linear_regression_height_class.eps"))
g.savefig(os.path.join(experiment_dir, "figures", f"{brain}_linear_regression_height_class.jpg"))